In [1]:
%cd ../

/home/qwj/code/HippoRAG


In [2]:
import os
import pickle

In [19]:
def load_file(file_path):
    if os.path.isfile(file_path):
        return pickle.load(open(file_path, 'rb'))

In [14]:
import itertools
def print_first10_dict(dict):
    first_ten_items = itertools.islice(dict.items(), 10)

    # 打印这些项
    for key, value in first_ten_items:
        print(f'{key}: {value}')

In [4]:
from src.hipporag_v3 import HipporagConfig

config = HipporagConfig(
    corpus_name='musique',
    extraction_model='openai',
    extraction_model_name='gpt-3.5-turbo-1106',
    graph_creating_retriever_name='facebook/contriever',
)

print('\n'.join(list(config.path_dict.values())))

output/musique_queries.named_entity_output.tsv
data/musique.json
data/musique_corpus.json
output/openie_musique_results_ner_gpt-3.5-turbo-1106_*.json
output/musique_facts_and_sim_graph_phrase_dict_ents_only_lower_preprocess_ner.v3.subset.p
output/musique_facts_and_sim_graph_fact_dict_ents_only_lower_preprocess_ner.v3.subset.p
output/musique_facts_and_sim_graph_relation_dict_ents_only_lower_preprocess_ner_facebook_contriever.v3.subset.p
output/musique_facts_and_sim_graph_doc_to_facts_ents_only_lower_preprocess_ner.v3.subset.p
output/musique_facts_and_sim_graph_facts_to_phrases_ents_only_lower_preprocess_ner.v3.subset.p
output/musique_facts_and_sim_graph_doc_to_facts_csr_ents_only_lower_preprocess_ner.v3.subset.p
output/musique_facts_and_sim_graph_facts_to_phrases_csr_ents_only_lower_preprocess_ner.v3.subset.p
output/musique_facts_and_sim_graph_mean_0.8_thresh_ents_only_lower_preprocess_ner_facebook_contriever.v3.subset.p
data/lm_vectors/facebook_contriever_mean/encoded_strings.txt
data/

In [20]:
graph_plus = load_file(config.get_path('graph_file'))
len(graph_plus)

349755

In [21]:
print_first10_dict(graph_plus)

(50797, 72087): 1.0
(72087, 50797): 1.0
(50797, 20270): 1.0
(20270, 50797): 1.0
(50797, 36493): 1.0
(36493, 50797): 1.0
(50797, 14273): 1.0
(14273, 50797): 1.0
(50797, 14755): 5.0
(14755, 50797): 5.0


In [24]:
kb_node_phrase_to_id = load_file(config.get_path('kb_node_phrase_to_id'))
len(kb_node_phrase_to_id)

91729

In [25]:
print_first10_dict(kb_node_phrase_to_id)

: 0
0: 1
0    0: 2
0    2: 3
0   1 972: 4
0   100: 5
0   435: 6
0   50: 7
0   6: 8
0   91: 9


In [27]:
relations_dict = load_file(config.get_path('relations_dict'))
len(relations_dict)

249395

In [28]:
print_first10_dict(relations_dict)

('lionel messi', 'royal spanish football federation'): enrolled in
('lionel messi', 'cesc f bregas'): befriended
('lionel messi', 'gerard piqu'): befriended
('lionel messi', 'baby dream team'): part of
('lionel messi', 'barcelona'): teammate of
('lionel messi', 'cadetes a'): top scorer for
('lionel messi', 'league'): won
('lionel messi', 'spanish cup'): won
('lionel messi', 'catalan cup'): won
('lionel messi', 'arsenal'): received offer from


In [85]:
import igraph as ig

g = ig.Graph()
print(g.summary())


IGRAPH U--- 0 0 -- 


In [86]:
g.add_vertices(len(kb_node_phrase_to_id)) 
g.vs['name'] = list(kb_node_phrase_to_id.keys())
g.vs['type'] = ['entity'] * len(kb_node_phrase_to_id)
print(f"Number of vertices: {g.vcount()}, Number of edges: {g.ecount()}, Is the graph directed: {g.is_directed()}")
print(g.summary())

Number of vertices: 91729, Number of edges: 0, Is the graph directed: False
IGRAPH UN-T 91729 0 -- 
+ attr: name (v), type (v)


In [87]:
print(g.summary())

IGRAPH UN-T 91729 0 -- 
+ attr: name (v), type (v)


In [88]:
g.add_edges(graph_plus.keys())

# 设置边的属性，例如权重g.summary()
g.es["weight"] = graph_plus.values()
print(f"Number of vertices: {g.vcount()}, Number of edges: {g.ecount()}, Is the graph directed: {g.is_directed()}")
print(g.summary())

Number of vertices: 91729, Number of edges: 349755, Is the graph directed: False
IGRAPH UNWT 91729 349755 -- 
+ attr: name (v), type (v), weight (e)


In [89]:
g.neighbors(100)

[6195, 6195, 75567, 75567]

In [90]:
g.vs[400]

igraph.Vertex(<igraph.Graph object at 0x7feba4fbc740>, 400, {'name': '1 july 2003', 'type': 'entity'})

In [94]:
g.neighbors(75567)

[100,
 100,
 3317,
 3317,
 3882,
 3882,
 20890,
 20890,
 21194,
 21194,
 23950,
 23950,
 35952,
 35952,
 35953,
 35953,
 36271,
 36271,
 40238,
 40238,
 42763,
 42763,
 57581,
 57581,
 64252,
 64252,
 64253,
 64253,
 66624,
 66624,
 69137,
 69137,
 75568,
 75568,
 79026,
 79026,
 80802,
 80802,
 90976,
 90976,
 91677,
 91677]

In [96]:
vertex = g.vs[75567]
print(f"{vertex['name']} (ID: {vertex.index})")
print()
for v_id in set(g.neighbors(75567)):
    print(f"{g.vs[v_id]['name']} (ID: {g.vs[v_id].index})")

sichuan (ID: 75567)

inland area (ID: 42763)
cool to very cold winters and mild summers (ID: 23950)
rainfall heavily concentrated in the summer (ID: 69137)
chengdu (ID: 20890)
zoige county (ID: 91677)
temperatures generally decrease with greater elevation (ID: 80802)
2008 sichuan earthquake (ID: 3882)
highly variable terrain (ID: 40238)
geologically active with landslides and earthquakes (ID: 36271)
sichuan province (ID: 75568)
strong monsoonal influences (ID: 79026)
poor health care (ID: 66624)
china s lowest sunshine totals (ID: 21194)
xichang (ID: 90976)
0 to 15  c (ID: 100)
mountainous areas producing a cooler but sunnier climate (ID: 57581)
garze county (ID: 35952)
garze county and zoige county (ID: 35953)
2 000 to 3 500 meters (ID: 3317)
panzhihua (ID: 64252)
panzhihua and xichang (ID: 64253)


In [21]:
import networkx as nx
import numpy as np

# Create a sample graph with 10 nodes
G = nx.DiGraph()
G.add_nodes_from(range(10))

# Adding some random edges (this is an example, you can change it as needed)
edges = [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 0), (0, 5), (2, 7), (3, 8)]
G.add_edges_from(edges)

# Define the reset distribution
reset_distribution = {i: 0.1 for i in range(10)}  # Reset probabilities for each vertex (uniform distribution)

# Define the damping factor
damping_factor = 0.85

# Calculate personalized PageRank with the reset distribution and damping factor
pagerank_scores = nx.pagerank(G, alpha=damping_factor, personalization=reset_distribution)

len(pagerank_scores)


10

In [19]:
import igraph as ig

# Create a sample graph with 10 nodes
g = ig.Graph()
g.add_vertices(10)
# Adding some random edges (this is an example, you can change it as needed)
g.add_edges([(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 0), (0, 5), (2, 7), (3, 8)])

# Define the reset distribution
reset_distribution = [0.1] * 10  # Reset probabilities for each vertex (uniform distribution)

# Define the damping factor
damping_factor = 0.85

# Calculate personalized PageRank with the reset distribution and damping factor
pagerank_scores = g.personalized_pagerank(damping=damping_factor, reset=reset_distribution, implementation='power')

len(pagerank_scores)


/tmp/ipykernel_973382/2516410642.py:16: DeprecationWarning: Partial string matches of enum members are deprecated since igraph 0.9.3; use strings that identify an enum member unambiguously.
  pagerank_scores = g.personalized_pagerank(damping=damping_factor, reset=reset_distribution, implementation='power')


10

In [3]:
from src.hipporag_v3 import HipporagConfig, HippoRAGv3
config = HipporagConfig(
    corpus_name='musique',
    extraction_model='openai',
    extraction_model_name='gpt-3.5-turbo-1106',
    graph_creating_retriever_name='colbertv2',
    damping=0.5,
)

/home/qwj/miniconda3/envs/hipporag/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import os

# Export environment variables
os.environ['OPENAI_API_KEY'] = 'aegsdvfhuijoak'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
hipporag = HippoRAGv3(config)

Augmenting Graph from Similarity


Building Graph: 100%|██████████| 394346/394346 [00:00<00:00, 591826.25it/s]
2024-07-26 12:34:53,119 - src.hipporag_v3 - INFO - Graph built: num vertices: 91729, num_edges: 394235




[Jul 26, 12:35:03] #> Note: Output directory data/lm_vectors/colbert/musique/phrase/indexes/nbits_2 already exists


[Jul 26, 12:35:13] #> Loading collection...
0M 
[Jul 26, 12:35:25] #> Loading codec...
[Jul 26, 12:35:25] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Jul 26, 12:35:25] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Jul 26, 12:35:25] #> Loading IVF...
[Jul 26, 12:35:25] #> Loading doclens...


100%|██████████| 4/4 [00:00<00:00, 1075.53it/s]

[Jul 26, 12:35:25] #> Loading codes and residuals...



100%|██████████| 4/4 [00:00<00:00, 290.94it/s]


In [12]:
entity = hipporag.kb.get('china')
print(entity, entity.in_relations, entity.out_relations)

Entity(name=china, id=21181, 67 in_relations, 27 out_relations defaultdict(<class 'list'>, {'continued war in': [Entity(name=japan, id=44665, 68 in_relations, 34 out_relations], 'borders': [Entity(name=myanmar, id=58235, 8 in_relations, 26 out_relations, Entity(name=chunggang county, id=21642, 1 in_relations, 6 out_relations], 'awarded broadcast rights in': [Entity(name=super sports, id=79562, 1 in_relations, 2 out_relations], 'of': [Entity(name=zhou dynasty, id=91615, 3 in_relations, 3 out_relations], 'is': [Entity(name=people s republic of china, id=65168, 6 in_relations, 3 out_relations], 'is from': [Entity(name=chen guangcheng, id=20870, 1 in_relations, 1 out_relations, Entity(name=zhang yibai, id=91588, 1 in_relations, 2 out_relations], 'ruled': [Entity(name=kuomintang, id=48593, 0 in_relations, 9 out_relations, Entity(name=yuan dynasty, id=91401, 6 in_relations, 9 out_relations, Entity(name=qing dynasty, id=68669, 6 in_relations, 22 out_relations], 'located in': [Entity(name=guan

In [5]:
hipporag.relations_dict

{('lionel messi', 'royal spanish football federation'): 'enrolled in',
 ('lionel messi', 'cesc f bregas'): 'befriended',
 ('lionel messi', 'gerard piqu'): 'befriended',
 ('lionel messi', 'baby dream team'): 'part of',
 ('lionel messi', 'barcelona'): 'teammate of',
 ('lionel messi', 'cadetes a'): 'top scorer for',
 ('lionel messi', 'league'): 'won',
 ('lionel messi', 'spanish cup'): 'won',
 ('lionel messi', 'catalan cup'): 'won',
 ('lionel messi', 'arsenal'): 'received offer from',
 ('cesc f bregas', 'england'): 'left for',
 ('gerard piqu', 'england'): 'left for',
 ('fc barcelona', '2006 07 season without trophies'): 'finished',
 ('fc barcelona', 'string of injuries to key players'): 'blamed for',
 ('fc barcelona', 'eto o'): 'injuries to',
 ('fc barcelona', 'lionel messi'): 'injuries to',
 ('eto o', 'frank rijkaard'): 'publicly criticized',
 ('eto o', 'barcelona'): 'leading scorer of',
 ('ronaldinho', 'lack of fitness affected his form'): 'admitted',
 ('barcelona', 'la liga'): 'were in 